# Arboles de Decisión


In [6]:
# Esto agrega al python path el directorio ..
import sys
sys.path.append("..")

import numpy as np
import config
from transformers import transformer
from data_builder import load_test_data, load_dev_data, load_small_dev_data


df, target = load_small_dev_data()


# Optimización de hiperparámetros

Busquemos los mejores (posibles) hiperparámetros

Para eso, primero veamos qué hiperparámetros nos provee la implementación de SKLearn.

In [8]:
X = transformer.fit_transform(df)

print X.shape

(8099, 150)



Algunas observaciones previas:

El árbol que se genera no es usando ID3 sino CART. Una pequeña introducción
se puede ver en

http://scikit-learn.org/stable/modules/tree.html#tree-algorithms-id3-c4-5-c5-0-and-cart

No está muy completo el asunto ahí, habría que hurgar algún paper extra

Opciones:

- criterion: gini o entropy. Son dos medidas distintas
- splitter: No estoy muy seguro
- max_depth: máxima profundidad del árbol
- max_features: qué porcentaje de variables tomo a la hora de partir un nodo.
  Estas variables se eligen aleatoriamente
- min_samples_split: cuántos elementos tengo que tener en un nodo para decidir
  partirlo

In [9]:
from search import find_best_classifier
from sklearn.tree import DecisionTreeClassifier

options = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': range(2, 52, 8),
    'max_features': np.arange(0.1, 1.0, 0.2),
    'min_samples_split': range(2, 102, 10),
}

classifier = find_best_classifier(
    X, target, DecisionTreeClassifier, param_distributions=options, 
    scoring='roc_auc', cv=10, n_jobs=6, n_iter=10)    

Buscando parámetros para DecisionTreeClassifier
Mejor combinación: {'min_samples_split': 92, 'splitter': 'best', 'criterion': 'entropy', 'max_depth': 50, 'max_features': 0.30000000000000004}
Mejor valor: 0.97895658442




In [10]:
from helpers import get_scores

get_scores(classifier, transformer)

,precision_score,accuracy_score,f1_score,recall_score,roc_auc_score
DecisionTreeClassifier,0.939087,0.934111,0.933736,0.928444,0.934111


In [11]:
df_test, target_test = load_test_data()



In [13]:
x_test = transformer.transform(df.payload)
y_test = target

KeyError: 'payload'